In [1]:
import requests
import json
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime as dt
import psycopg2

In [2]:
chicago_crimes = pd.read_csv("./Resources/Chicago_Crimes_2001_to_Present.csv", sep=',')
chicago_crimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [4]:
chicago_UE = pd.read_csv("./Resources/Chicago_UE_annual.csv", sep=',')
chicago_UE = chicago_UE.drop(columns=['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'])
chicago_UE.head()

,Year,Labor Force,Labor Force Participation Rate,Employed,Employment Participation Rate,Unemployed,Unemployment Rate,Illinois Rate
0,2020.0,"3,617,500",63.0,"3,244,300",56.5,"373,300",10.3,9.5
1,2019.0,"3,725,500",64.7,"3,582,900",62.2,"142,600",3.8,4.0
2,2018.0,"3,733,500",64.7,"3,582,400",62.1,"151,100",4.0,4.4
3,2017.0,"3,768,300",65.2,"3,581,800",62.0,"186,500",4.9,5.0
4,2016.0,"3,806,000",65.9,"3,580,000",61.9,"226,000",5.9,5.9


In [5]:
chicago_UE.dropna()

,Year,Labor Force,Labor Force Participation Rate,Employed,Employment Participation Rate,Unemployed,Unemployment Rate,Illinois Rate
0,2020.0,"3,617,500",63.0,"3,244,300",56.5,"373,300",10.3,9.5
1,2019.0,"3,725,500",64.7,"3,582,900",62.2,"142,600",3.8,4.0
2,2018.0,"3,733,500",64.7,"3,582,400",62.1,"151,100",4.0,4.4
3,2017.0,"3,768,300",65.2,"3,581,800",62.0,"186,500",4.9,5.0
4,2016.0,"3,806,000",65.9,"3,580,000",61.9,"226,000",5.9,5.9
5,2015.0,"3,774,200",65.3,"3,546,100",61.4,"228,100",6.0,6.0
6,2014.0,"3,782,800",65.6,"3,506,500",60.8,"276,300",7.3,7.2
7,2013.0,"3,806,700",66.2,"3,450,500",60.0,"356,300",9.4,9.1
8,2012.0,"3,786,400",66.3,"3,432,500",60.1,"353,900",9.3,9.0
9,2011.0,"3,727,900",65.7,"3,345,600",58.9,"382,300",10.3,9.8


In [6]:
chicago_crimes = chicago_crimes.drop(columns=['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Location Description', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code', 'Updated On'])
chicago_crimes.head()

,Primary Type,Description,Arrest,Domestic,X Coordinate,Y Coordinate,Year,Latitude,Longitude,Location
0,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,1165074.0,1875917.0,2015,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,THEFT,POCKET-PICKING,False,False,1138875.0,1904869.0,2015,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,THEFT,OVER $500,False,True,NaN,NaN,2018,NaN,NaN,NaN
3,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,1152037.0,1920384.0,2015,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,ASSAULT,SIMPLE,False,True,1141706.0,1900086.0,2015,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [7]:
chicago_crimes = chicago_crimes.set_index(['Year'])

In [8]:
chicago_crimes = chicago_crimes.dropna()

In [9]:
primary_type = chicago_crimes['Primary Type'].unique()

In [10]:
chicago_crimes.sort_index(inplace=True)

In [11]:
crime_count = chicago_crimes.groupby('Year')['Primary Type'].count()

In [73]:
crime_count_dict = {'Crimes_Committed': crime_count}
crime_count_df = pd.DataFrame(crime_count_dict)
crime_count_df.sort_index(ascending=False)
crime_count_df.reset_index()
crime_count_df

,Crimes_Committed
Year,
2001,482877
2002,471528
2003,472038
2004,467202
2005,449920
2006,445547
2007,435693
2008,420033
2009,386048


In [100]:
#crime_count_df = crime_count_df.reset_index()
#crime_count_df = crime_count_df.drop(columns=['level_0', 'index'])
crime_count_df = crime_count_df.set_index(['Date'])

In [102]:
#crime_count_df = crime_count_df[crime_count_df.Date < 2021]
crime_count_df

,Crimes_Committed
Date,
2020,208461
2019,259136
2018,263450
2017,264939
2016,267244
2015,257906
2014,273731
2013,306365
2012,335451


In [103]:
#crime_count_df =  crime_count_df.set_index('Date')
#crime_count_df = crime_count_df.sort_index(ascending=False)
crime_count_df

,Crimes_Committed
Date,
2020,208461
2019,259136
2018,263450
2017,264939
2016,267244
2015,257906
2014,273731
2013,306365
2012,335451


In [108]:
#chicago_UE = chicago_UE[chicago_UE.Date >= 2001]
chicago_UE = chicago_UE.drop(columns=['Labor Force', 'Labor Force Participation Rate', 'Employed', 'Employment Participation Rate', 'Unemployed', 'Illinois Rate'])
chicago_UE

,Unemployment Rate
Year,
2020.0,10.3
2019.0,3.8
2018.0,4.0
2017.0,4.9
2016.0,5.9
2015.0,6.0
2014.0,7.3
2013.0,9.4
2012.0,9.3


In [125]:
#chicago_UE.reset_index()
#chicago_UE = chicago_UE.rename(columns={'Year':'Date'})
#chicago_UE_rates = chicago_UE['Unemployment Rate']
chicago_UE.reset_index()

,Year,Unemployment Rate
0,2020.0,10.3
1,2019.0,3.8
2,2018.0,4.0
3,2017.0,4.9
4,2016.0,5.9
5,2015.0,6.0
6,2014.0,7.3
7,2013.0,9.4
8,2012.0,9.3
9,2011.0,10.3


In [130]:
#chicago_UE.reset_index()
#chicago_UE = chicago_UE.rename(columns={'Year':'Date'})
#chicago_UE_rates = chicago_UE['Unemployment Rate']
#chicago_UE.reset_index().rename(columns={'Year': 'Date'})
chicago_UE.reset_index().rename(columns={'Year': 'Date', 'Unemployment Rate':'Unemployment_rate'})

,Date,Unemployment_rate
0,2020.0,10.3
1,2019.0,3.8
2,2018.0,4.0
3,2017.0,4.9
4,2016.0,5.9
5,2015.0,6.0
6,2014.0,7.3
7,2013.0,9.4
8,2012.0,9.3
9,2011.0,10.3


In [112]:
UE_dict = {'Unemployement_Rate': chicago_UE_rates}
UE_df = pd.DataFrame(UE_dict)

In [113]:
combined_df = pd.merge(crime_count_df, UE_df, left_index=True, right_index=True)
combined_df

,Crimes_Committed,Unemployement_Rate
2020,208461,10.3
2019,259136,3.8
2018,263450,4.0
2017,264939,4.9
2016,267244,5.9
2015,257906,6.0
2014,273731,7.3
2013,306365,9.4
2012,335451,9.3
2011,351310,10.3


In [131]:
connection = "postgres:postgres@localhost:5432/ETL-project"
engine = create_engine(f'postgresql://{connection}')

In [ ]:
crime_count_df.to_sql(name='Crime', con=engine, if_exists='append', index=False)

In [ ]:
chicago_UE.to_sql(name='Unemployment', con=engine, if_exists='append', index=False)

In [56]:
combined_df.to_csv('chicago_crime_UE')